# Install dependencies

In [25]:
!pip3 install pretty_midi bokeh mlflow google-cloud-storage cloudstorage

    100% |████████████████████████████████| 51kB 3.5MB/s 
  Running setup.py bdist_wheel for inflection ... done
  Stored in directory: /home/jupyter/.cache/pip/wheels/9f/5a/d3/6fc3bf6516d2a3eb7e18f9f28b472110b59325f3f258fe9211
Successfully built inflection


# Fetch and clean data

In [23]:
import pandas as pd
from tqdm import tqdm_notebook
import pretty_midi
import os
import shutil

In [1]:
!wget -P ../data https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip

--2019-06-30 12:37:23--  https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.166.128, 2a00:1450:400c:c0a::80
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.166.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59243107 (56M) [application/zip]
Saving to: ‘../data/maestro-v2.0.0-midi.zip’

maestro-v2.0.0-midi 100%[===================>]  56.50M  64.9MB/s    in 0.9s    

2019-06-30 12:37:25 (64.9 MB/s) - ‘../data/maestro-v2.0.0-midi.zip’ saved [59243107/59243107]



In [ ]:
!unzip ../data/maestro-v2.0.0-midi.zip -d ../data

In [5]:
metadata = pd.read_csv('../data/maestro-v2.0.0/maestro-v2.0.0.csv')

In [6]:
len(metadata)

1282

In [14]:
songs = []
for _, row in tqdm_notebook(list(metadata.iterrows())):
    midi = pretty_midi.PrettyMIDI(os.path.join('../data/maestro-v2.0.0', row['midi_filename']))
    midi.split = row['split']
    songs.append(midi)

In [18]:
shutil.rmtree('../data/songs.txt', ignore_errors=True)
with open('../data/songs.txt', 'w') as f:
    for midi in tqdm_notebook(songs):
        assert len(midi.instruments) == 1
        f.write('{split}#{tempo}#{notes}\n'.format(
            split=midi.split, tempo=midi.estimate_tempo(), 
            notes=[(n.start, n.end, n.pitch, n.velocity) for n in midi.instruments[0].notes]
        ))